# Gmail API - Jupyter Notebook
Envío, gestión y eliminación de correos usando Gmail API desde Jupyter Notebook

## 1. Instalación de dependencias

In [16]:
!pip install google-auth-oauthlib google-auth-httplib2 google-api-python-client python-dotenv

## 2. Configuración e imports

In [1]:
import os
import base64
import json
import time
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

 

# Scopes de la API (añadido modify para eliminar)
SCOPES = [
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/gmail.readonly'
]

print("✅ Módulos importados y configuración cargada")

✅ Módulos importados y configuración cargada


## 3. Función de autenticación

In [2]:
def load_credentials():
    """Carga o genera credenciales para la API de Gmail"""
    creds = None
    
    # El archivo token.json almacena los tokens de acceso y actualización
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # Si no hay credenciales válidas, permite que el usuario inicie sesión
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Para uso en CLI con refresh token preexistente
            refresh_token = os.getenv('GMAIL_REFRESH_TOKEN')
            client_id = os.getenv('GOOGLE_CLIENT_ID')
            client_secret = os.getenv('GOOGLE_CLIENT_SECRET')
            
            if refresh_token and client_id and client_secret:
                creds = Credentials(
                    token=None,
                    refresh_token=refresh_token,
                    token_uri='https://oauth2.googleapis.com/token',
                    client_id=client_id,
                    client_secret=client_secret,
                    scopes=SCOPES
                )
                # Refrescar el token
                creds.refresh(Request())
            else:
                # Flujo de autenticación interactivo
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
            
            # Guarda las credenciales para la próxima ejecución
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
    
    return creds

def get_gmail_service():
    """Obtiene el servicio de Gmail"""
    creds = load_credentials()
    return build('gmail', 'v1', credentials=creds)

In [3]:
get_gmail_service()

## 4. Funciones auxiliares

In [4]:
def create_message(to, subject, body):
    """Crea un mensaje MIME para enviar"""
    message = MIMEText(body, 'plain', 'utf-8')
    message['to'] = to
    message['subject'] = subject
    return {
        'raw': base64.urlsafe_b64encode(
            message.as_bytes()
        ).decode('ascii')
    }

def decode_message_body(data):
    """Decodifica el cuerpo del mensaje desde base64"""
    if 'data' in data:
        return base64.urlsafe_b64decode(data['data']).decode('utf-8')
    return ''

def get_message_text_parts(parts):
    """Extrae el texto de las partes del mensaje"""
    text_parts = []
    for part in parts:
        if part['mimeType'] == 'text/plain':
            text_parts.append(decode_message_body(part['body']))
        elif part['mimeType'] == 'text/html':
            text_parts.append(decode_message_body(part['body']))
        elif 'parts' in part:
            # Mensajes multipart
            text_parts.extend(get_message_text_parts(part['parts']))
    return text_parts

print("✅ Funciones auxiliares definidas")

✅ Funciones auxiliares definidas


## 5. Función para enviar correos

In [5]:
def send_email(to=None, subject=None, body=None):
    """Envía un correo usando Gmail API"""
    try:
        service = get_gmail_service()
        if not service:
            print("❌ No se pudo obtener el servicio de Gmail")
            return None
        
        # Valores por defecto
        to = to or os.getenv('GMAIL_TO', 'jlaybar@dominio.com')
        subject = subject or os.getenv('GMAIL_SUBJECT', 'Prueba desde Jupyter Notebook')
        body = body or os.getenv('GMAIL_BODY', 'Hola! 👋 Este correo fue enviado desde Jupyter Notebook usando la API de Gmail.')
        
        # Crear y enviar mensaje
        message = create_message(to, subject, body)
        result = service.users().messages().send(
            userId='me', 
            body=message
        ).execute()
        
        print(f'✅ Correo enviado exitosamente!')
        print(f'   ID: {result["id"]}')
        print(f'   Para: {to}')
        print(f'   Asunto: {subject}')
        
        return result
        
    except HttpError as err:
        error_detail = err.content.decode('utf-8') if err.content else str(err)
        print(f'❌ Error enviando correo: {error_detail}')
        return None
    except Exception as err:
        print(f'❌ Error inesperado: {str(err)}')
        return None

print("✅ Función de envío definida")

✅ Función de envío definida


## 6. Función para listar mensajes

In [6]:
def list_messages(max_results=10, label_ids='INBOX'):
    """Lista los últimos mensajes del buzón"""
    try:
        service = get_gmail_service()
        if not service:
            print("❌ No se pudo obtener el servicio de Gmail")
            return None
        
        # Obtener lista de mensajes
        result = service.users().messages().list(
            userId='me',
            maxResults=max_results,
            labelIds=label_ids
        ).execute()
        
        messages = result.get('messages', [])
        messages_list = []
        
        print(f"📧 Obteniendo detalles de {len(messages)} mensajes...")
        
        # Obtener detalles de cada mensaje
        for i, msg in enumerate(messages):
            print(f"   Procesando mensaje {i+1}/{len(messages)}...")
            
            message_detail = service.users().messages().get(
                userId='me', 
                id=msg['id'],
                format='metadata',
                metadataHeaders=['Subject', 'From', 'Date']
            ).execute()
            
            # Extraer headers importantes
            headers = {h['name']: h['value'] for h in message_detail.get('payload', {}).get('headers', [])}
            
            messages_list.append({
                'id': msg['id'],
                'threadId': msg.get('threadId'),
                'snippet': message_detail.get('snippet', ''),
                'subject': headers.get('Subject', ''),
                'from': headers.get('From', ''),
                'date': headers.get('Date', ''),
                'labelIds': message_detail.get('labelIds', [])
            })
        
        print(f"✅ Se obtuvieron {len(messages_list)} mensajes")
        return messages_list
        
    except HttpError as err:
        print(f'❌ Error de Gmail API: {err}')
        return None
    except Exception as err:
        print(f'❌ Error inesperado: {str(err)}')
        return None

print("✅ Función para listar mensajes definida")

✅ Función para listar mensajes definida


## 7. Función para obtener mensaje específico

In [7]:
def get_message_details(message_id):
    """Obtiene el cuerpo completo de un mensaje específico"""
    try:
        service = get_gmail_service()
        if not service:
            print("❌ No se pudo obtener el servicio de Gmail")
            return None
        
        # Obtener mensaje completo
        message = service.users().messages().get(
            userId='me', 
            id=message_id,
            format='full'
        ).execute()
        
        # Extraer headers
        headers = {h['name']: h['value'] for h in message.get('payload', {}).get('headers', [])}
        
        # Extraer cuerpo del mensaje
        body_text = ''
        payload = message.get('payload', {})
        
        if 'parts' in payload:
            # Mensaje multipart
            text_parts = get_message_text_parts(payload['parts'])
            body_text = ''.join(text_parts)
        elif 'body' in payload and 'data' in payload['body']:
            # Mensaje simple
            body_text = decode_message_body(payload['body'])
        
        message_details = {
            'id': message['id'],
            'threadId': message.get('threadId'),
            'snippet': message.get('snippet', ''),
            'subject': headers.get('Subject', ''),
            'from': headers.get('From', ''),
            'to': headers.get('To', ''),
            'date': headers.get('Date', ''),
            'body': body_text,
            'labelIds': message.get('labelIds', [])
        }
        
        print(f"✅ Mensaje obtenido exitosamente")
        return message_details
        
    except HttpError as err:
        print(f'❌ Error de Gmail API: {err}')
        return None
    except Exception as err:
        print(f'❌ Error inesperado: {str(err)}')
        return None

print("✅ Función para obtener mensaje definida")

✅ Función para obtener mensaje definida


## 8. Función para buscar mensajes

In [8]:
def search_messages(search_query, max_results=100):
    """Busca mensajes basados en una consulta"""
    try:
        service = get_gmail_service()
        if not service:
            print("❌ No se pudo obtener el servicio de Gmail")
            return None
        
        result = service.users().messages().list(
            userId='me',
            q=search_query,
            maxResults=max_results
        ).execute()
        
        messages = result.get('messages', [])
        messages_list = []
        
        print(f"🔍 Buscando mensajes con query: '{search_query}'")
        print(f"📨 Encontrados {len(messages)} mensajes")
        
        # Obtener detalles de cada mensaje
        for i, msg in enumerate(messages):
            message_detail = service.users().messages().get(
                userId='me', 
                id=msg['id'],
                format='metadata',
                metadataHeaders=['Subject', 'From', 'Date']
            ).execute()
            
            # Extraer headers importantes
            headers = {h['name']: h['value'] for h in message_detail.get('payload', {}).get('headers', [])}
            
            messages_list.append({
                'id': msg['id'],
                'threadId': msg.get('threadId'),
                'snippet': message_detail.get('snippet', ''),
                'subject': headers.get('Subject', ''),
                'from': headers.get('From', ''),
                'date': headers.get('Date', ''),
                'labelIds': message_detail.get('labelIds', [])
            })
        
        return messages_list
        
    except HttpError as err:
        print(f'❌ Error buscando mensajes: {err}')
        return []

print("✅ Función de búsqueda definida")

✅ Función de búsqueda definida


## 9. 🆕 Función para eliminar mensajes por palabra clave

In [9]:
def delete_messages_by_keyword(keyword, search_in='both', max_results=500, dry_run=False):
    """
    Elimina mensajes que contengan una palabra clave en asunto o remitente
    
    Args:
        keyword (str): Palabra clave a buscar
        search_in (str): Dónde buscar ('subject', 'from', 'both')
        max_results (int): Máximo número de mensajes a procesar
        dry_run (bool): Si es True, solo muestra qué se eliminaría sin hacer cambios
    """
    try:
        service = get_gmail_service()
        if not service:
            print("❌ No se pudo obtener el servicio de Gmail")
            return None
        
        # Construir query de búsqueda
        search_queries = []
        
        if search_in in ['subject', 'both']:
            search_queries.append(f'subject:"{keyword}"')
        
        if search_in in ['from', 'both']:
            search_queries.append(f'from:"{keyword}"')
        
        # Buscar mensajes para cada query
        all_messages = []
        seen_ids = set()
        
        for query in search_queries:
            messages = search_messages(query, max_results)
            for msg in messages:
                if msg['id'] not in seen_ids:
                    seen_ids.add(msg['id'])
                    all_messages.append(msg)
        
        if not all_messages:
            print("✅ No se encontraron mensajes que coincidan con la búsqueda")
            return None
        
        # Mostrar mensajes encontrados
        print(f"\n📋 MENSAJES ENCONTRADOS ({len(all_messages)}):")
        print("=" * 100)
        for i, msg in enumerate(all_messages, 1):
            source = []
            if search_in in ['subject', 'both'] and keyword.lower() in msg['subject'].lower():
                source.append('asunto')
            if search_in in ['from', 'both'] and keyword.lower() in msg['from'].lower():
                source.append('remitente')
            
            print(f"{i}. ID: {msg['id']}")
            print(f"   De: {msg['from']}")
            print(f"   Asunto: {msg['subject']}")
            print(f"   Fecha: {msg['date']}")
            print(f"   Coincidencia en: {', '.join(source)}")
            print(f"   Snippet: {msg['snippet'][:100]}...")
            print("-" * 100)
        
        if dry_run:
            print(f"\n🔍 MODO SIMULACIÓN: Se eliminarían {len(all_messages)} mensajes")
            print("   Ejecuta con dry_run=False para eliminar realmente")
            return all_messages
        
        # Confirmar eliminación
        print(f"\n⚠️  ¿Estás seguro de que quieres eliminar {len(all_messages)} mensajes?")
        confirm = input("   Escribe 'ELIMINAR' para confirmar: ")
        if confirm != 'ELIMINAR':
            print("❌ Eliminación cancelada")
            return None
        
        # Eliminar mensajes
        deleted_count = 0
        error_count = 0
        
        for i, msg in enumerate(all_messages, 1):
            try:
                service.users().messages().delete(
                    userId='me',
                    id=msg['id']
                ).execute()
                deleted_count += 1
                print(f"🗑️  Eliminado {i}/{len(all_messages)}: {msg['subject'][:50]}...")
                
                # Pequeña pausa para evitar rate limits
                time.sleep(0.1)
                
            except HttpError as err:
                error_count += 1
                print(f"❌ Error eliminando mensaje {msg['id']}: {err}")
                continue
        
        print(f"\n📊 RESULTADO DE ELIMINACIÓN:")
        print(f"   ✅ Eliminados: {deleted_count}")
        print(f"   ❌ Errores: {error_count}")
        print(f"   📊 Total: {len(all_messages)}")
        
        return {
            'deleted': deleted_count,
            'errors': error_count,
            'total': len(all_messages)
        }
        
    except Exception as err:
        print(f'❌ Error inesperado: {str(err)}')
        return None

print("✅ Función de eliminación definida")

✅ Función de eliminación definida


## 10. Ejemplos de uso

### 10.1 Enviar un correo de prueba

In [27]:
# Enviar correo de prueba
result = send_email(
    to="destinatario@ejemplo.com",  # Cambia por un email real
    subject="Prueba desde Jupyter Notebook",
    body="¡Hola! Este es un correo de prueba enviado desde Jupyter Notebook usando Gmail API."
)

if result:
    print("Correo enviado con éxito!")
else:
    print("Error al enviar el correo")

✅ Correo enviado exitosamente!
   ID: 19a11ddcb8072ad1
   Para: destinatario@ejemplo.com
   Asunto: Prueba desde Jupyter Notebook
Correo enviado con éxito!


### 10.2 Listar últimos mensajes

In [25]:
# Listar últimos 5 mensajes
messages = list_messages(max_results=5)

if messages:
    print("\n📬 Últimos mensajes:")
    print("-" * 80)
    for i, msg in enumerate(messages):
        print(f"{i+1}. De: {msg['from']}")
        print(f"   Asunto: {msg['subject']}")
        print(f"   Fecha: {msg['date']}")
        print(f"   ID: {msg['id']}")
        print(f"   Snippet: {msg['snippet'][:100]}...")
        print("-" * 80)

📧 Obteniendo detalles de 5 mensajes...
   Procesando mensaje 1/5...
   Procesando mensaje 2/5...
   Procesando mensaje 3/5...
   Procesando mensaje 4/5...
   Procesando mensaje 5/5...
✅ Se obtuvieron 5 mensajes

📬 Últimos mensajes:
--------------------------------------------------------------------------------
1. De: Mail Delivery Subsystem <mailer-daemon@googlemail.com>
   Asunto: Delivery Status Notification (Failure)
   Fecha: Thu, 23 Oct 2025 09:15:08 -0700 (PDT)
   ID: 19a11da42641c0bd
   Snippet: No se ha encontrado la dirección Tu mensaje no se ha entregado a destinatario@ejemplo.com porque no ...
--------------------------------------------------------------------------------
2. De: jlaybar@gmail.com
   Asunto: 
   Fecha: Thu, 23 Oct 2025 10:41:22 -0500
   ID: 19a11bb5cf857033
   Snippet: Contenido del email...
--------------------------------------------------------------------------------
3. De: jlaybar@gmail.com
   Asunto: 
   Fecha: Thu, 23 Oct 2025 10:24:16 -0500
   ID: 1

### 10.3 Buscar mensajes específicos

In [13]:
# Buscar mensajes con una palabra clave
palabra_clave = "API"

search_results = search_messages(palabra_clave, max_results=5)

if search_results:
    print("\n🔍 Resultados de búsqueda:")
    for i, msg in enumerate(search_results, 1):
        print(f"{i}. {msg['from']} - {msg['subject']}")

🔍 Buscando mensajes con query: 'API'
📨 Encontrados 5 mensajes

🔍 Resultados de búsqueda:
1. Mail Delivery Subsystem <mailer-daemon@googlemail.com> - Delivery Status Notification (Failure)
2. jlaybar@gmail.com - 
3. jlaybar@gmail.com - Asunto del correo2 enviado desde FastAPI con OAuth2 y Gmail API
4. jlaybar@gmail.com - 
5. Mail Delivery Subsystem <mailer-daemon@googlemail.com> - Delivery Status Notification (Failure)


### 10.4 🆕 Simular eliminación de mensajes (MODO SEGURO)

In [14]:
# Simular eliminación de mensajes con "spam" en el asunto
print("🔍 MODO SIMULACIÓN - No se eliminará nada")
deletion_simulation = delete_messages_by_keyword(
    keyword="Revolut", 
    search_in="subject", 
    max_results=10,
    dry_run=True  # ¡MODO SEGURO!
)

🔍 MODO SIMULACIÓN - No se eliminará nada
🔍 Buscando mensajes con query: 'subject:"Revolut"'
📨 Encontrados 1 mensajes

📋 MENSAJES ENCONTRADOS (1):
1. ID: 19a0fdae420b4579
   De: Revolut <no-reply@revolut.com>
   Asunto: Recibe 60 € por cada persona que se una a Revolut
   Fecha: Thu, 23 Oct 2025 06:56:34 +0000
   Coincidencia en: asunto
   Snippet: ¿Quieres llevarte 60 €? Invita amigos a Revolut y llévate una recompensa por cada uno que se registr...
----------------------------------------------------------------------------------------------------

🔍 MODO SIMULACIÓN: Se eliminarían 1 mensajes
   Ejecuta con dry_run=False para eliminar realmente


In [12]:
# Simular eliminación de mensajes con "spam" en el asunto
print("🔍 MODO SIMULACIÓN - No se eliminará nada")
deletion_simulation = delete_messages_by_keyword(
    keyword="Facebook", 
    search_in="from",  
    max_results=10,
    dry_run=True  # ¡MODO SEGURO!
)

🔍 MODO SIMULACIÓN - No se eliminará nada
🔍 Buscando mensajes con query: 'from:"Facebook"'
📨 Encontrados 10 mensajes

📋 MENSAJES ENCONTRADOS (10):
1. ID: 19007f5f254a86d1
   De: Facebook <notification@facebookmail.com>
   Asunto: Sugerencias para ti de Eurosport.
   Fecha: Tue, 11 Jun 2024 08:41:17 -0700
   Coincidencia en: remitente
   Snippet: Mira esta foto de Eurosport. Hola, Jaime: Explora contenido sugerido de Eurosport. Eurosport compart...
----------------------------------------------------------------------------------------------------
2. ID: 19004134ee5b132a
   De: Facebook <friendupdates@facebookmail.com>
   Asunto: ⏰ Fernando Cid Lozano acaba de publicar una historia que caduca en 24 horas
   Fecha: Mon, 10 Jun 2024 14:34:45 -0700
   Coincidencia en: remitente
   Snippet: The story will expire mañana a las 9:20 pm, so don&#39;t miss out on the opportunity to see it. Mira...
----------------------------------------------------------------------------------------------------

### 10.5 🆕 Eliminación real de mensajes (¡CUIDADO!)

In [14]:
# ⚠️ ELIMINACIÓN REAL - Descomenta solo si estás seguro

# Eliminar mensajes de un remitente específico
deletion_result = delete_messages_by_keyword(
    keyword="Facebook", 
    search_in="from", 
    max_results=100,
    dry_run=False  # ⚠️ Esto eliminará permanentemente
)

if deletion_result:
    print(f"Eliminación completada: {deletion_result}")

print("⚠️  Celda de eliminación real comentada por seguridad")

🔍 Buscando mensajes con query: 'from:"Facebook"'
📨 Encontrados 100 mensajes

📋 MENSAJES ENCONTRADOS (100):
1. ID: 19004134ee5b132a
   De: Facebook <friendupdates@facebookmail.com>
   Asunto: ⏰ Fernando Cid Lozano acaba de publicar una historia que caduca en 24 horas
   Fecha: Mon, 10 Jun 2024 14:34:45 -0700
   Coincidencia en: remitente
   Snippet: The story will expire mañana a las 9:20 pm, so don&#39;t miss out on the opportunity to see it. Mira...
----------------------------------------------------------------------------------------------------
2. ID: 19003b1854d57462
   De: Carolina en Facebook <friendupdates@facebookmail.com>
   Asunto: Carolina Butrón agregó un nuevo video
   Fecha: Mon, 10 Jun 2024 12:48:00 -0700
   Coincidencia en: remitente
   Snippet: Carolina Butrón agregó un nuevo video. 10 de junio a la 1:25 pm Ver video Anabel Butron de Pedro y 1...
----------------------------------------------------------------------------------------------------
3. ID: 19002ca4eaae7c

### 10.6 Obtener mensaje específico

In [18]:
# Obtener detalles de un mensaje específico (usa un ID de la lista anterior)
if 'messages' in locals() and messages:
    message_id = messages[0]['id']  # Usar el primer mensaje de la lista
    message_details = get_message_details(message_id)
    
    if message_details:
        print(f"\n📧 Mensaje: {message_details['subject']}")
        print("=" * 80)
        print(f"De: {message_details['from']}")
        print(f"Para: {message_details['to']}")
        print(f"Fecha: {message_details['date']}")
        print(f"ID: {message_details['id']}")
        print("-" * 80)
        print("CUERPO:")
        print("-" * 80)
        print(message_details['body'][:500] + "..." if len(message_details['body']) > 500 else message_details['body'])
else:
    print("Primero ejecuta la celda de listar mensajes para obtener un ID válido")

✅ Mensaje obtenido exitosamente

📧 Mensaje: Asunto del correo2 enviado desde FastAPI con OAuth2 y Gmail API
De: jlaybar@gmail.com
Para: jlaybar@gmail.com
Fecha: Thu, 23 Oct 2025 09:31:05 -0500
ID: 19a117b02cb6a2c0
--------------------------------------------------------------------------------
CUERPO:
--------------------------------------------------------------------------------
Este es el cuerpo del correo enviado desde la aplicación FastAPI con OAuth2 y Gmail API.


## 11. Función para mostrar resumen del buzón

In [ ]:
def mailbox_summary():
    """Muestra un resumen del buzón de Gmail"""
    try:
        service = get_gmail_service()
        if not service:
            return None
        
        # Obtener perfiles
        profile = service.users().getProfile(userId='me').execute()
        
        # Obtener conteo de mensajes por etiqueta
        labels = service.users().labels().list(userId='me').execute()
        
        print("📊 RESUMEN DEL BUZÓN")
        print("=" * 50)
        print(f"Email: {profile['emailAddress']}")
        print(f"Mensajes totales: {profile.get('messagesTotal', 'N/A')}")
        print(f"Hilos totales: {profile.get('threadsTotal', 'N/A')}")
        print("\nEtiquetas principales:")
        
        for label in labels.get('labels', [])[:10]:  # Mostrar solo las primeras 10
            print(f"  • {label['name']}: {label.get('messagesTotal', 0)} mensajes")
        
        return profile
        
    except Exception as err:
        print(f"❌ Error obteniendo resumen: {str(err)}")
        return None

# Ejecutar resumen
mailbox_summary()

## 12. Configuración de variables de entorno

Crea un archivo `.env` en la misma carpeta con:

```env
GOOGLE_CLIENT_ID=tu_client_id_aqui
GOOGLE_CLIENT_SECRET=tu_client_secret_aqui
GMAIL_REFRESH_TOKEN=tu_refresh_token_aqui
GMAIL_TO=destinatario@ejemplo.com
GMAIL_SUBJECT=Asunto por defecto
GMAIL_BODY=Cuerpo por defecto
```

O configura las variables directamente:

In [ ]:
# Configuración directa (alternativa a .env)
os.environ['GOOGLE_CLIENT_ID'] = 'tu_client_id_aqui'
os.environ['GOOGLE_CLIENT_SECRET'] = 'tu_client_secret_aqui'
os.environ['GMAIL_REFRESH_TOKEN'] = 'tu_refresh_token_aqui'

print("✅ Variables de entorno configuradas")

## Resumen de funciones disponibles

- `send_email(to, subject, body)` - Enviar correo
- `list_messages(max_results, label_ids)` - Listar mensajes
- `get_message_details(message_id)` - Obtener mensaje específico
- `search_messages(search_query, max_results)` - Buscar mensajes
- `delete_messages_by_keyword(keyword, search_in, max_results, dry_run)` - Eliminar mensajes por palabra clave
- `mailbox_summary()` - Resumen del buzón

## 🆕 Características de eliminación

1. **Búsqueda flexible**: Busca en asunto, remitente o ambos
2. **Modo simulación**: `dry_run=True` para ver qué se eliminaría
3. **Confirmación**: Requiere escribir "ELIMINAR" para proceder
4. **Proceso por lotes**: Maneja grandes cantidades eficientemente
5. **Reporte detallado**: Muestra resultados de la eliminación

**⚠️ PRECAUCIÓN**: La eliminación de mensajes es permanente. Siempre usa `dry_run=True` primero.